In [6]:
import os
import sys
import time

import dlib
import boto3
import cv2
from IPython.display import Image
import numpy as np

from app import utils
from app.image_helpers import bgr2png
from app.video_streams import stream_factory
from app.yolo_model import YOLOv5Model

import requests
requests.packages.urllib3.disable_warnings() 

ModuleNotFoundError: No module named 'dlib'

In [2]:
STREAM_URL = 'http://192.168.8.219:8080/'
WEIGHTS = '20210916-184902-kube%3Aadmin-weights.pt'
COMPUTE_DEVICE = 'cuda:0'  # cpu or cuda:x

In [3]:
# initialize video stream
stream = stream_factory(STREAM_URL)
# initialize detection model
s3_client = boto3.client('s3', endpoint_url='https://' + os.environ.get('S3_ENDPOINT_URL'), verify=False)
model = YOLOv5Model(class_map=['car'], conf_thresh=0.3, device=COMPUTE_DEVICE)
model.from_s3(s3_client, os.environ.get('S3_BUCKET_NAME'), WEIGHTS)
model.imgsz = 416
# initialize centroid trackers
trackers = []

In [4]:
# process each frame
for (timestamp, frame) in stream:
    if frame is None:
        continue
    
    with utils.BlockTimer('Run inference on frame... ') as t:
        detections = model(frame)
    
    break

Starting Run inference on frame... ... done [0.028236 seconds]


In [5]:
detections

[{'box': [130.0, 112.0, 152.0, 128.0],
  'confidence': 0.7574032545089722,
  'class': 'car'},
 {'box': [160.0, 105.0, 178.0, 117.0],
  'confidence': 0.6601072549819946,
  'class': 'car'},
 {'box': [267.0, 81.0, 273.0, 87.0],
  'confidence': 0.6427752375602722,
  'class': 'car'},
 {'box': [180.0, 97.0, 194.0, 109.0],
  'confidence': 0.6031914353370667,
  'class': 'car'},
 {'box': [201.0, 83.0, 211.0, 91.0],
  'confidence': 0.5668219327926636,
  'class': 'car'},
 {'box': [212.0, 88.0, 223.0, 97.0],
  'confidence': 0.496455579996109,
  'class': 'car'},
 {'box': [190.0, 95.0, 205.0, 106.0],
  'confidence': 0.3665356934070587,
  'class': 'car'},
 {'box': [205.0, 90.0, 216.0, 98.0],
  'confidence': 0.3629326820373535,
  'class': 'car'}]

In [ ]:
for detection in detections:
    confidence = detection['confidence']
    label = detection['class']
    box = detections['box']

    tracker = dlib.correlationable, it will be called once for each serialized storage with two arguments: storage and location. The storage argument will be the initial deserialization of the stor_tracker()
    rect = dlib.rectangle(*box)
    tracker.start_track(rgb, rect)
    trackers.append(tracker)

    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
    cv2.putText(frame, label, (box[0], box[1] - 15),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

In [ ]:
Image(data=bgr2png(frame)) 